In [15]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import sys
import glob
import math
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import models
import torch.optim as optim
import torch.nn.functional as F
train_root = "/home/yulin/Documents/DLCV/hw2/hw2_train_val/train15000/"
test_root = "/home/yulin/Documents/DLCV/hw2/hw2_train_val/val1500/"
if torch.cuda.is_available():
    model.cuda()

In [4]:
class MYDATASET(Dataset):
    def __init__(self, root, transform=None):
        """ Intialize the MNIST dataset """
        self.images = None
        self.labels = None
        self.filenames = []
        self.root = root
        self.transform = transform
        #     parent = os.path.join(path, i)　　read filenames
        path = root+"images/"
        path_ = root+"labelTxt_hbb/"
        self.filenames=sorted(glob.glob(os.path.join(path,'*.jpg')))
        self.txtnames=sorted(glob.glob(os.path.join(path_,'*.txt')))
        self.len = len(self.filenames)
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        image_fn = self.filenames[index]
        temp = Image.open(image_fn).convert('RGB')
        image = temp.resize((448, 448),Image.ANTIALIAS)    
        f = open(self.txtnames[index])
        line = f.readline()
        list = []
        while line:
            a = line.split( )
            b = a[0:]
            list.append(b)
            line = f.readline()
        f.close
        label_one=np.zeros((7,7,26))
        for s in range(len(list)):
            x=0.5*(float(list[s][2])+float(list[s][0]))/512
            y=0.5*(float(list[s][5])+float(list[s][1]))/512
            w=1*(float(list[s][2])-float(list[s][0]))/512
            h=1*(float(list[s][5])-float(list[s][1]))/512
            class_=list[s][8]
            diff_=list[s][9]
            i=math.floor(7*x)
            j=math.floor(7*y)
            if label_one[i][j][0]==0:
                
                label_one[i][j][0]=x
                label_one[i][j][1]=y
                label_one[i][j][2]=w
                label_one[i][j][3]=h
                label_one[i][j][4]=1
                if class_==str('plane'):
                    label_one[i][j][10]=1
                if class_==str('ship'):
                    label_one[i][j][11]=1
                if class_==str('storage-tank'):
                    label_one[i][j][12]=1
                if class_==str('baseball-diamond'):
                    label_one[i][j][13]=1
                if class_==str('tennis-court'):
                    label_one[i][j][14]=1
                if class_==str('basketball-court'):
                    label_one[i][j][15]=1
                if class_==str('ground-track-field'):
                    label_one[i][j][16]=1
                if class_==str('harbor'):
                    label_one[i][j][17]=1
                if class_==str('bridge'):
                    label_one[i][j][18]=1
                if class_==str('small-vehicle'):
                    label_one[i][j][19]=1
                if class_==str('large-vehicle'):
                    label_one[i][j][20]=1
                if class_==str('helicopter'):
                    label_one[i][j][21]=1
                if class_==str('roundabout'):
                    label_one[i][j][22]=1
                if class_==str('soccer-ball-field'):
                    label_one[i][j][23]=1
                if class_==str('swimming-pool'):
                    label_one[i][j][24]=1
                if class_==str('container-crane'):
                    label_one[i][j][25]=1
            else:
                pass
            label = torch.from_numpy(label_one).type(torch.float)
            

        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return self.len

In [5]:
trainset=MYDATASET(root=train_root, transform=transforms.ToTensor())
testset=MYDATASET(root=test_root, transform=transforms.ToTensor())

In [6]:
#print('# images in trainset:', len(trainset)) # Should print 15000
#print('# images in testset:', len(testset)) # Should print 1500

In [7]:
trainset_loader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=1)
testset_loader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=1)
# get some random training images
dataiter = iter(trainset_loader)
images, labels = dataiter.next()

print('Image tensor in each batch:', images.shape, images.dtype)
print('Label tensor in each batch:', labels.shape, labels.dtype)

Image tensor in each batch: torch.Size([1, 3, 448, 448]) torch.float32
Label tensor in each batch: torch.Size([1, 7, 7, 26]) torch.float32


In [8]:
def judge(inputs):
    s=7
    index=[]
    for i in range(s):
        for j in range(s):
            if labels[0][i][j][0] == 0:
                pass
            else:
                index.append([i,j])
    return index

In [9]:
model = models.Yolov1_vgg16bn(pretrained=True)

In [10]:
#ssd=model(images)
#ssd.shape

In [11]:
#index=judge(labels)
def iou(inputs, targets, index):
    cross=[]
    pr=[]
    confidence=[]
    for i,j in index:
        for e in range(2):
            squ_label=targets[0][i][j][0:4]
            squ_model=inputs[0][i][j][0+e:4+e]
            x_min1=squ_label[0]-0.5*squ_label[2]
            x_max1=squ_label[0]+0.5*squ_label[2]
            y_min1=squ_label[1]-0.5*squ_label[3]
            y_max1=squ_label[1]+0.5*squ_label[3]
            x_min2=squ_model[0]-0.5*squ_model[2]
            x_max2=squ_model[0]+0.5*squ_model[2]
            y_min2=squ_model[1]-0.5*squ_model[3]
            y_max2=squ_model[1]+0.5*squ_model[3]

            ww=min(x_max1,x_max2)-max(x_min1,x_min2)
            hh=min(y_max1,y_max2)-max(x_min1,y_min2)
            if ww<=0 or hh<=0:
                cross.append([0])
            else:
                cross.append([ww*hh])
    for k in range(len(index)):
        z=max(cross[0+2*k:2+2*k])
        if cross[0+2*k]>=cross[1+2*k]:
            confidence.append(cross[0+2*k]) 
            pr.append([0])
        else:
            confidence.append(cross[1+2*k]) 
            pr.append([1])
    confidence=torch.FloatTensor(confidence)
  

    return confidence,pr
#confidence,pr=iou(ssd,labels,index)

#print(confidence)
#print(pr)

In [12]:
class LossFunc(nn.Module):
    def __init__(self):
        super(LossFunc, self).__init__()   
        #number
        s=7
        ss=s*s
        coord=5
        noodj=0.5    
    
    def forward(self, inputs, targets):
        s=7
        ss=s*s
        coord=5
        noodj=0.5
        index=judge(targets)
        delta_sum=0
        for i,j in index:
            temp=inputs[:, i, j, 1:3]-targets[:, i, j, 1:3]
            delta_xy1=torch.sum(torch.pow(temp,2))
            temp2=torch.sqrt(inputs[:, i, j, 3:5])-torch.sqrt(labels[:, i, j, 3:5])
            delta_wh1=torch.sum(torch.pow(temp2,2))
            temp3=inputs[:, i, j, 6:8]-targets[:, i, j, 1:3]
            delta_xy2=torch.sum(torch.pow(temp3,2))
            temp4=torch.sqrt(inputs[:, i, j, 8:10])-torch.sqrt(labels[:, i, j, 3:5])
            delta_wh2=torch.sum(torch.pow(temp4,2))
            delta_sum = delta_sum+delta_wh1+delta_xy1+delta_wh2+delta_xy2
        first=coord*delta_sum#中心點誤差
        
        confidence,pr=iou(inputs, targets, index)#獲得置信度與概率
        hat_c=torch.FloatTensor(len(pr),2).zero_()
        m=0
        second=0
        for i,j in index:
            hat_c[m][0]=inputs[:, i, j, 4:5]
            hat_c[m][1]=inputs[:, i, j, 9:10]
            m=m+1
        key=[i for i,v in enumerate(pr) if v==[1]]
        for s in key:
            hat_c[s,:]=hat_c[s,[1,0]]
            tar_1=torch.sum(torch.pow(hat_c[:,0]-torch.squeeze(confidence),2))
            tar_2=torch.sum(torch.pow(hat_c[:,1],2))
            
            second=second+tar_1+noodj*tar_2
        
        third=0
        for i,j in index:
            obj=torch.squeeze(targets[:, i, j, 10:])
            a=torch.eq(obj,1)
            b=(a==1).nonzero()
            for o in range(len(b)):
                c=b[o].item()
                d=inputs[:, i, j,10+c:11+c]
                e=torch.pow(d-torch.FloatTensor([[1]]),2)
                third=third+e
            
        loss=first+second+third
        return loss

In [13]:
def train(model, epoch, log_interval=100):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion=LossFunc()
    model.train()  # Important: set training mode
    
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(trainset_loader):
            #data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1


In [ ]:
def train_some(model, epoch, log_interval=100):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion=LossFunc()
    model.train()  # Important: set training mode
    
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(trainset_loader):
            #data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1

In [14]:
#train(model, 5)

In [16]:
def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [17]:
def train_save(model, epoch, save_interval, log_interval=100):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion=LossFunc()
    model.train()
    
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(trainset_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            if iteration % save_interval == 0 and iteration > 0:
                save_checkpoint('mnist-%i.pth' % iteration, model, optimizer)
            iteration += 1
    
    # save the final model
    save_checkpoint('mnist-%i.pth' % iteration, model, optimizer)

In [ ]:
# create a brand new model
model = Net().to(device)
test(model)
train_save(model, 5, 500, 100)